[<img src="../header.svg">](../index.ipynd)

---

# Simulation of Drug Transport across a Virtual Skin Membrane

### Initialize UG4 (for 2D and standard algebra)

In [1]:
import modsimtools as util

* Initializing: paths... done, bridge... done, plugins... done                 *
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/Poroelasticity/lua/lua-include.lua
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/d3f/lua/lua-include.lua
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/Limex/lua/lua-include.lua
LIMEX-Plugin: Loading util.limex.*
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/RepoTrend/lua/lua-include.lua
RepoTrend-Plugin: Extended package.path:./?.lua;/usr/local/share/lua/5.1/?.lua;/usr/local/share/lua/5.1/?/init.lua;/usr/local/lib/lua/5.1/?.lua;/usr/local/lib/lua/5.1/?/init.lua;/Users/anaegel/Software/ug4-git/plugins/RepoTrend/lua/?.lua


### Create Domain

In [2]:
requiredSubsets = {"LIP", "COR", "BOTTOM_SC", "TOP_SC"}
gridName = "skin2d-aniso.ugx"
numRefs = 2

In [3]:
dom = util.CreateDomain(gridName, numRefs, requiredSubsets)

Loading Domain skin2d-aniso.ugx ... done.
Performing integrity check on domain ... done.
Refining(2): 1 2 done.


### Create Approximation space

In [4]:
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct("c","Lagrange", 1) 
approxSpace:add_fct("b","Lagrange", 1) -- NEW: Two functions are required
approxSpace:init_levels()
approxSpace:init_top_surface()
print("Approximation space:")
approxSpace:print_statistic()

Approximation space:
| ----------------------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                               |
|  Algebra: Block 1 (divide by 1 for #Index)                                                |
|                                                                                           |
|    GridLevel   |       Domain |       0: LIP |       1: COR | 2: BOTTOM_SC |    3: TOP_SC |
| ----------------------------------------------------------------------------------------- |
| (lev,    0)    |         1360 |           64 |         1216 |           40 |           40 |
| (lev,    1)    |         5226 |         1566 |         3504 |           78 |           78 |
| (lev,    2)    |        20482 |         8734 |        11440 |          154 |          154 |
| (lev,    0, g) |         1360 |           64 |         1216 |           40 |           40 |
| (lev,    1, g) |         5226 |      

## Create a convection-diffusion-equation

$$ \partial_t c + \nabla \cdot [-D \nabla c] + k_f \cdot c - k_r \cdot b = 0 $$
$$ \partial_t b   - k_f \cdot c + k_r \cdot b = 0 $$

with steady-state: 
$ b = \frac{k_f}{k_r} c $

* Define model parameters

In [5]:
D={
    ["LIP"] = 1, 
    ["COR"] = 0.01, 
}

kf = 2.0
kr = 1.0

* Print characteristic times

In [6]:
L=17.6 -- characteristic length
print("Characteristic times:")
print("Binding (c->b):\t\t"..1.0/kf)
print("Binding (b->c):\t\t"..1.0/kr)
print("Diffusion [LIP]:\t"..L*L/D["LIP"])
print("Diffusion [COR]:\t"..L*L/D["COR"])

Characteristic times:
Binding (c->b):		0.5
Binding (b->c):		1
Diffusion [LIP]:	309.76
Diffusion [COR]:	30976


* The system requires **separate** element discs **for each species**:

In [7]:
elemDiscC ={}

elemDiscC["LIP"] = ConvectionDiffusion("c", "LIP", "fv1")
elemDiscC["LIP"]:set_diffusion(D["LIP"])
elemDiscC["LIP"]:set_mass_scale(1.0)

elemDiscC["COR"] = ConvectionDiffusion("c", "COR", "fv1")
elemDiscC["COR"]:set_diffusion(D["COR"])
elemDiscC["COR"]:set_mass_scale(1.0)

In [8]:
elemDiscB ={}

elemDiscB["LIP"] = ConvectionDiffusion("b", "LIP", "fv1")
elemDiscB["LIP"]:set_diffusion(0.0)
elemDiscB["LIP"]:set_mass_scale(1.0)

elemDiscB["COR"] = ConvectionDiffusion("b", "COR", "fv1")
elemDiscB["COR"]:set_diffusion(0.0)
elemDiscB["COR"]:set_mass_scale(1.0)

* Add corneocyte binding, i.e., reactions in corneocytes

In [9]:
myReactionC = kf*elemDiscC["COR"]:value()-kr*elemDiscB["COR"]:value()
myReactionB = -kf*elemDiscC["COR"]:value()+kr*elemDiscB["COR"]:value()

elemDiscC["COR"]:set_reaction(myReactionC)
elemDiscB["COR"]:set_reaction(myReactionB)

* Boundary conditions:

In [10]:
dirichletBnd = DirichletBoundary()
dirichletBnd:add(1.0, "c", "TOP_SC")
dirichletBnd:add(0.0, "c", "BOTTOM_SC")
dirichletBnd:add(0.0, "b", "TOP_SC") -- NEW
dirichletBnd:add(0.0, "b", "BOTTOM_SC")

* Summarize everything in domain discretization:

In [11]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDiscC["LIP"]) 
domainDisc:add(elemDiscC["COR"])

-- NEW: More discs
domainDisc:add(elemDiscB["LIP"])
domainDisc:add(elemDiscB["COR"])
domainDisc:add(dirichletBnd)

## Create  solver

In [12]:
lsolver=SuperLU()

## Solve transient problem
* Define parameters for time stepping:

In [13]:
startTime = 0.0
endTime = 500.0
dt=endTime/100.0

* Define grid function:

In [14]:
u = GridFunction(approxSpace)
u:set(0.0)

* Solve problem (using lua util):

In [15]:
util.SolveLinearTimeProblem(u, domainDisc, lsolver, VTKOutput(), "vtk/bindingLo",
                            "ImplEuler", 1, startTime, endTime, dt); 

SolveLinearTimeProblem, Linear Solver setup:
SuperLU
>> Writing start values
++++++ TIMESTEP 1 BEGIN (current time: 0) ++++++
++++++ Time step size: 5
++++++ Assembling Matrix/Rhs for step size 5
++++++ TIMESTEP 1 END   (current time: 5) ++++++
++++++ TIMESTEP 2 BEGIN (current time: 5) ++++++
++++++ Time step size: 5
++++++ TIMESTEP 2 END   (current time: 10) ++++++
++++++ TIMESTEP 3 BEGIN (current time: 10) ++++++
++++++ Time step size: 5
++++++ TIMESTEP 3 END   (current time: 15) ++++++
++++++ TIMESTEP 4 BEGIN (current time: 15) ++++++
++++++ Time step size: 5
++++++ TIMESTEP 4 END   (current time: 20) ++++++
++++++ TIMESTEP 5 BEGIN (current time: 20) ++++++
++++++ Time step size: 5
++++++ TIMESTEP 5 END   (current time: 25) ++++++
++++++ TIMESTEP 6 BEGIN (current time: 25) ++++++
++++++ Time step size: 5
++++++ TIMESTEP 6 END   (current time: 30) ++++++
++++++ TIMESTEP 7 BEGIN (current time: 30) ++++++
++++++ Time step size: 5
++++++ TIMESTEP 7 END   (current time: 35) ++++++
++++++

## Alternative
* Use C++-Objects

In [16]:
local timeDisc=ThetaTimeStep(domainDisc, 1.0)
timeInt = ConstStepLinearTimeIntegrator(timeDisc)
timeInt:set_linear_solver(lsolver)
timeInt:set_time_step(dt)

In [17]:
local vtkObserver = VTKOutputObserver("vtk/BindingSol.vtk", VTKOutput())
timeInt:attach_finalize_observer(vtkObserver)

In [18]:
timeInt:apply(u, endTime, u, startTime)

+++ Integrating: [	0	, 	500	] with dt=	5(100 iters)
+++ Const timestep +++1(t=0, dt=5)
+++ Assemble (t=0, dt=5)
+++ Const timestep +++2(t=5, dt=5)
+++ Const timestep +++3(t=10, dt=5)
+++ Const timestep +++4(t=15, dt=5)
+++ Const timestep +++5(t=20, dt=5)
+++ Const timestep +++6(t=25, dt=5)
+++ Const timestep +++7(t=30, dt=5)
+++ Const timestep +++8(t=35, dt=5)
+++ Const timestep +++9(t=40, dt=5)
+++ Const timestep +++10(t=45, dt=5)
+++ Const timestep +++11(t=50, dt=5)
+++ Const timestep +++12(t=55, dt=5)
+++ Const timestep +++13(t=60, dt=5)
+++ Const timestep +++14(t=65, dt=5)
+++ Const timestep +++15(t=70, dt=5)
+++ Const timestep +++16(t=75, dt=5)
+++ Const timestep +++17(t=80, dt=5)
+++ Const timestep +++18(t=85, dt=5)
+++ Const timestep +++19(t=90, dt=5)
+++ Const timestep +++20(t=95, dt=5)
+++ Const timestep +++21(t=100, dt=5)
+++ Const timestep +++22(t=105, dt=5)
+++ Const timestep +++23(t=110, dt=5)
+++ Const timestep +++24(t=115, dt=5)
+++ Const timestep +++25(t=120, dt=5)
+++ 